# CASTOR UVMOS Spectrograph - DQ & DZ White Dwarfs Resolution Comparison 
This code was designed to rotate through the DQ and DZ spectra for each temperature and abundance while calculating the SNR and Exposure Time for a given wavelength. It then creates a table and saves it into an excel file. 

The code can also run through a ranges of resolutions by simply changing the MySpec.dispersion value. To adjust it, simply enter the desired dispersions into the resultion list and run the code as usual. 

In [27]:
import astropy.units as u
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy import constants as const
import math
import pandas as pd
import csv

from castor_etc.background import Background
from castor_etc.photometry import Photometry
from castor_etc.sources import ExtendedSource, GalaxySource, PointSource
from castor_etc.telescope import Telescope

from castor_etc.uvmos_spectroscopy import UVMOS_Spectroscopy

In [29]:
def read_spectra(spectral_type, temp, abundance):
    
    """ 
    Definition
    ----------
    Reads in the parameters needed to pick a synthetic spectra.
    
    Parameters
    ----------
    spectral_type: str
        The name corresponding to the stellar type (either DZ or DQ) you'd like to view.   
    temp: int
        The temperature of the desired stellar spectra.
    abundance: float
        The abudance of the desired stellar spectra.
        
    Returns
    -------
    The wavelength and flux tables corresponding to the specified stellar type, temperature and abundance.
    
    Notes
    -----
    - This function accesses the read_DQ and read_DZ functions to open the requested synthetic spectra. 
    - This function was simply designed to allow easy access to the synthetic spectra and as a result is particular to these examples. 
    - Adjustments will need to be made if files are in different locations.
    
    """
    
    if spectral_type == "DZ":
        
        try:
            file_name = f'{spectral_type} Stars/8000{int(temp/100)}hepu_g14.11_{spectral_type}_{abundance}'
            file = open(file_name, 'r')
        except:
            file_name = f'{spectral_type} Stars/800{int(temp/100)}hepu_g14.11_{spectral_type}_{abundance}'
            file = open(file_name, 'r')
        
        return read_DZ(file)
            
    elif spectral_type == "DQ":
        if abundance < 8.5:
            try:
                file_name = f'{spectral_type} Stars/8000{int(temp/100)}hepu_g14.12_{spectral_type}_{abundance}'
                file = open(file_name, 'r')
            except:
                file_name = f'{spectral_type} Stars/800{int(temp/100)}hepu_g14.12_{spectral_type}_{abundance}'
                file = open(file_name, 'r')
        else:
            try:
                file_name = f'{spectral_type} Stars/8000{int(temp/100)}hepu_g15.12_{spectral_type}_{abundance}'
                file = open(file_name, 'r')
            except:
                file_name = f'{spectral_type} Stars/800{int(temp/100)}hepu_g15.12_{spectral_type}_{abundance}'
                file = open(file_name, 'r')
                
        return read_DQ(file)
    
def read_DZ(file):
    
    """ 
    Definition
    ----------
    Reads in the synthetic spectra for DZ White Dwarf Stars.
    
    Parameters
    ----------
    file: str
        The file for the corresponding DZ spectrum. 
        
    Returns
    -------
    The wavelength and flux tables listed in the file.
    
    Notes
    -----
    This function only handles the DZ White Dwarf Stars and as a result will break if used on a DQ spectra. 
    To access the information stored in the DQ White Dwarf files please use the read_DQ function. 
    
    """

    Lines = file.readlines()[1:]
    del Lines[6200]
    
    # Changing values to floats
    for i in range(len(Lines)):
        
        temp = Lines[i].split(" ")
        temp = list(filter(None, temp))
        
        for e in range(len(temp)):
            temp[e] = float(temp[e])  
            
        Lines[i] = temp
        
    w, flux = sum(Lines[:6200], []), sum(Lines[6200:], [])
    
    return np.array(w)*u.AA, np.array(flux)

def read_DQ(file):
    
    """ 
    Definition
    ----------
    Reads in the synthetic spectra for DQ White Dwarf Stars.
    
    Parameters
    ----------
    file: str
        The file corresponding to the DQ spectrum. 
        
    Returns
    -------
    The wavelength and flux tables listed in the file.
    
    Notes
    -----
    This function only handles the DQ White Dwarf Stars and as a result will break if used on a DZ spectra. 
    To access the information stored in the DZ White Dwarf files please use the read_DZ function. 
    
    """
    
    Lines = file.readlines()[1:]
    del Lines[5500]
    
    # Changing values to floats
    for i in range(len(Lines)):
        
        temp = Lines[i].split(" ")
        temp = list(filter(None, temp))
        
        for e in range(len(temp)):
            temp[e] = float(temp[e]) 
            
        Lines[i] = temp
        
    w, flux = sum(Lines[:5500], []), sum(Lines[5500:], [])
    
    return np.array(w)*u.AA, np.array(flux)

# Additional Libraries Required to use calc_snr_for_region_from_t 
from specutils.manipulation import extract_region
from specutils import SpectralRegion, Spectrum1D
import scipy

def calc_snr_for_region_from_t(t, wave, region=None, nread=1):
    
    """
    Definition
    ----------
    Calculation of the signal-to-noise ratio for a given region taking into account the noise produced by the telescope. 
    
    Parameters
    -----------
    t: int or float
        The Exposure Time to be used when calculating the signal-to-noise ratio.
    wave: int or float
        The central wavelength that you would like the signal-to-noise ratio calculated for.
    region: list
        The maximum and minimum wavelengths (without units) corresponding to the region over which to calculate the signal-to-noise ratio.
    nread: int
    
    Returns
    -------
    The signal-to-noise ratio for a specified wavelength over a specific region. 
        
    """
    
    spec = Spectrum1D(spectral_axis=MySpec.waves_CASTORSpectrum*u.AA, flux=MySpec.source_CASTORSpectrum*u.Unit('erg cm-2 s-1 AA-1'))
    Bspec = Spectrum1D(spectral_axis=MySpec.waves_CASTORSpectrum*u.AA, flux=(MySpec.background_CASTORSpectrum)*u.Unit('erg cm-2 s-1 AA-1'))
    
    if region is not None:
        
        regionMin, regionMax = region
        
        calc_spectrum = extract_region(spec, SpectralRegion(regionMin*u.AA, regionMax*u.AA))
        back_spectrum = extract_region(Bspec, SpectralRegion(regionMin*u.AA, regionMax*u.AA))

        read_npix = MySpec.source_extracted_numpixs
        signal_t = np.array(calc_spectrum.flux)*t
        noise = signal_t + t*read_npix*(np.array(back_spectrum.flux)+MySpec.TelescopeObj.dark_current) + read_npix*MySpec.TelescopeObj.read_noise**2*nread 
        S_N = np.divide(signal_t, np.sqrt(noise), where=np.sqrt(noise) > 0 )
        interp_SN = scipy.interpolate.interp1d(calc_spectrum.spectral_axis, S_N, kind='cubic')

        return interp_SN(wave)
        
    else:  
        calc_spectrum = MySpec.waves_CASTORSpectrum 
        
        return MySpec.calc_snr_from_t(t, wave)

# Additional libraries required to use Equivalent Width
import warnings
from astropy.modeling import models
from specutils.fitting import fit_generic_continuum
from specutils.analysis import equivalent_width

from math import floor, ceil

def EquivalentWidth(region, t, wave, b=10):
    
    """
    Definition
    ----------
    Fitting of a continuum for a given region followed by a calculation for the equivalent width and its error
    
    Parameters
    -----------
    region: list
        The maximum and minimum wavelengths (without units) corresponding to the region over which to calculate the equivalent width and its error. 
    t: int or float
        The Exposure Time to be used when calculating the signal-to-noise ratio.
    wave: int or float
        The central wavelength that you would like the signal-to-noise ratio calculated for.
    b: int
        A integer value which is used to expand the spectral region to calculate the continuum. For example, if b equals 10, 
        then the region will be expanded to the nearest ten above and below the maximum and minimum wavelengths 
        respectively (if the wavelength region is [1987.52, 1997.49] then the expanded region would be [1980, 2000] if b equals 10).
    
    Returns
    -------
    The calculated Equivalent Width and its error. Also plots the spectrum and the continuum for review. 
    
    Notes
    -----
    - The degree of the Chebyshev function used when creating the continuum (using 'fit_generic_continuum') depends on b. 
        The best fit when b = 10 is to set the Chebyshev degree to 4 but depending on the size of the expanded region
        used to create the continuum, a different degree can be a better fit.
        
    """
    
    if region[0] > region[1]:
        region.reverse() 

    regionMin, regionMax = region
    calc_regionMin, calc_regionMax = [floor(regionMin/b)*b, ceil(regionMax/b)*b]

    spec = Spectrum1D(spectral_axis=MySpec.waves_CASTORSpectrum*u.AA, flux=MySpec.source_CASTORSpectrum*u.Unit('erg cm-2 s-1 AA-1'))
    calc_spec = extract_region(spec, SpectralRegion(calc_regionMin*u.AA, calc_regionMax*u.AA))

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        fit = fit_generic_continuum(calc_spec, model=models.Chebyshev1D(degree=4), exclude_regions=[SpectralRegion(regionMin*u.AA, regionMax*u.AA)])
    
    continuum = fit(calc_spec.spectral_axis)

    normalizedSpec = (np.array(calc_spec.flux))/(np.array(continuum))
    normalizedSpec = Spectrum1D(spectral_axis=calc_spec.spectral_axis, flux=normalizedSpec*u.Unit('erg cm-2 s-1 AA-1'))

    # Calculate the equivalent Width
    EquivWidth = equivalent_width(normalizedSpec, 1, regions=SpectralRegion(regionMin*u.AA, regionMax*u.AA))

    # Calculate the error is the equivalent width
    EquivWidthError = (1+(np.mean(continuum)/np.mean(calc_spec.flux))) * ((regionMax*u.AA - regionMin*u.AA - EquivWidth)/calc_snr_for_region_from_t(t, wave, [regionMin, regionMax]))

    f, ax = plt.subplots()  
    ax.plot(calc_spec.spectral_axis, calc_spec.flux, label='Source Spectrum')  
    ax.plot(calc_spec.spectral_axis, continuum, label='Fitted Continuum')  
    ax.set_title("CASTOR Spectrum and Fitted Continuum")  
    ax.set_xlabel('Wavelength '+r'[$\AA$]')
    ax.set_ylabel('Flux [counts / second ]')
    plt.legend()
    ax.grid(True)  

    f, ax = plt.subplots()  
    ax.plot(normalizedSpec.spectral_axis, normalizedSpec.flux)  
    ax.set_title("Normalized Spectrum")  
    ax.set_xlabel('Wavelength '+r'[$\AA$]')
    ax.set_ylabel('Flux [counts / second ]')
    ax.grid(True) 
    
    return EquivWidth, EquivWidthError

In [21]:
resolution = [0.061] 
MyTelescope = Telescope()
MyBackground = Background()

Resolutions = []
Temperatures = []
Abundances = []
Wavelengths = []
SNRs = []
ExpTimes = []

Resolutions1 = []
Temperatures1 = []
Abundances1 = []
Wavelengths1 = []
SNRs1 = []
ExpTimes1 = []

for r in resolution:
        
    # Temperatures from: 6,500 K to 9,500 K for DQ Stars
    for temp in range(6500, 10000, 500):
        if temp >= 8000:
            abundances = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5]
        else:
            abundances = [4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5]
        for a in abundances:
            MySource = PointSource()
            
            spectral_type="DQ"
            wavelength, flux = read_spectra(spectral_type, temp, a)
    
            MySource.set_spectrum(wavelengths = wavelength, spectrum = flux, unit = 'fnu')
            MySource.norm_to_AB_mag(10)

            MySpec = UVMOS_Spectroscopy(MyTelescope, MySource, MyBackground)
            
            MySpec.dispersion = r * u.nm
            
            MySpec.specify_slit()

            MySpec.calc_source_CASTORSpectrum(extraction_width=1, extraction_lowerlim=4, extraction_upperlim = 7)
            MySpec.calc_background_CASTORSpectrum(extraction_width=1, extraction_lowerlim=4, extraction_upperlim = 7)
    
            # Calculate the SNR and Exposure Times for approximate wavelengths
            if temp <= 7000:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 8.0 or a == 7.5:
                    wavelengths = [2478.77]
                elif a == 7.0: 
                    wavelengths = [2298.33, 2478.77]
                elif a == 6.5: 
                    wavelengths = [2287.12, 2298.33, 2478.77]
                elif a == 6.0: 
                    wavelengths = [1993.81, 2287.12, 2298.33, 2478.77]
                elif a == 5.5: 
                    wavelengths = [1993.81, 2287.12, 2298.33, 2409.61, 2478.77]
                elif a == 5.0: 
                    wavelengths = [1993.81, 2409.61, 2478.77]
                elif a == 4.5 or a == 4.0: 
                    wavelengths = [1993.81, 2409.61, 2478.77, 2583.08, 2967.40]
            elif temp == 7500:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 8.0 or a == 7.5 or a == 7.0:
                    wavelengths = [2478.77]
                elif a == 6.5: 
                    wavelengths = [1993.81, 2298.33, 2478.77]
                elif a == 6.0 or a == 5.5 or a == 5.0: 
                    wavelengths = [1993.81, 2287.12, 2298.33, 2478.77]
                elif a == 4.5 or a == 4.: 
                    wavelengths = [1993.81, 2409.61, 2478.77, 2583.08, 2967.40]
            elif temp == 8000:
                if a == 9.5 or a == 9.0:
                    wavelengths = [1931.02]
                elif a == 8.5 or a == 8.0:
                    wavelengths = [1931.02, 2478.77]
                elif a == 7.5 or a == 7.0:
                    wavelengths = [1751.98, 2478.77]
                elif a == 6.5:
                    wavelengths = [1751.98, 1993.81, 2478.77]
                elif a == 6.0: 
                    wavelengths = [1993.81, 2298.33, 2478.77]
                elif a == 5.5 or a == 5.0: 
                    wavelengths = [1993.81, 2287.12, 2298.33, 2478.77]
                elif a <= 4.5: 
                    wavelengths = [1993.81, 2409.61, 2478.77, 2583.08, 2967.40]
            elif temp == 8500:
                if a == 9.5 or a == 9.0:
                    wavelengths = [1931.02]
                elif a == 8.5 or a == 8.0:
                    wavelengths = [1931.02, 2478.77]
                elif a == 7.5 or a == 7.0:
                    wavelengths = [1751.98, 2478.77]
                elif a == 6.5 or a == 6.0:
                    wavelengths = [1751.98, 1993.81, 2478.77]
                elif a == 5.5: 
                    wavelengths = [1993.81, 2298.33, 2478.77]
                elif a == 5.0 or a == 4.5: 
                    wavelengths = [1993.81, 2287.12, 2298.33, 2478.77]
                elif a <= 4.0: 
                    wavelengths = [1993.81, 2409.61, 2478.77, 2583.08, 2967.40]
            elif temp == 9000:
                if a == 9.5 or a == 9.0:
                    wavelengths = [1931.02]
                elif a == 8.5 or a == 8.0:
                    wavelengths = [1931.02, 2478.77]
                elif a == 7.5 or a == 7.0:
                    wavelengths = [1751.98, 2478.77]
                elif a == 6.5 or a == 6.0 or a == 5.5:
                    wavelengths = [1751.98, 1993.81, 2478.77]
                elif a == 5.0: 
                    wavelengths = [1993.81, 2298.33, 2478.77, 2583.08]
                elif a == 4.5: 
                    wavelengths = [1993.81, 2287.12, 2298.33, 2478.77, 2583.08]
                elif a <= 4.0: 
                    wavelengths = [1993.81, 2409.61, 2478.77, 2583.08, 2967.40]
            elif temp == 9500:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 8.0 or a == 7.5 or a == 7.0:
                    wavelengths = [1931.02, 2478.77]
                elif a == 6.5 or a == 6.0 or a == 5.5 or a == 5.0:
                    wavelengths = [1751.98, 1993.81, 2478.77]
                    if a == 5.0: 
                        wavelengths.append(2583.08)
                elif a == 4.5 or a == 4.0: 
                    wavelengths = [1993.81, 2298.33, 2478.77, 2583.08]
                    if a == 4.0:
                        wavelengths.append(2967.40)
                elif a <= 3.5: 
                    wavelengths = [1993.81, 2409.61, 2478.77, 2583.08, 2967.40]
            for w in wavelengths:
                Resolutions.append(r)
                Temperatures.append(temp)
                Abundances.append(-a)
                Wavelengths.append(w)
                SNRs.append(np.round(float(MySpec.calc_snr_from_t(1000, w)), 3))
                ExpTimes.append(np.round((float(MySpec.calc_t_from_snr(30, w))), 3))
               
    # Temperatures from: 10,000 K to 15,000 K for DQ Stars
    for temp in range(10000, 16000, 1000):
        if temp == 10000:
            abundances = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5]
        else: 
            abundances = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 8.5, 9.0, 9.5]
        for a in abundances:
            MySource = PointSource()
    
            spectral_type="DQ"
            wavelength, flux = read_spectra(spectral_type, temp, a)
    
            MySource.set_spectrum(wavelengths = wavelength, spectrum = flux, unit = 'fnu')
            MySource.norm_to_AB_mag(10)

            MySpec = UVMOS_Spectroscopy(MyTelescope, MySource, MyBackground)
            
            MySpec.dispersion = r * u.nm
    
            MySpec.specify_slit()

            MySpec.calc_source_CASTORSpectrum(extraction_width=1, extraction_lowerlim=4, extraction_upperlim = 7)
            MySpec.calc_background_CASTORSpectrum(extraction_width=1, extraction_lowerlim=4, extraction_upperlim = 7)
    
            # Calculate the SNR for approximate wavelengths
            if temp == 10000:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 8.0 or a == 7.5 or a == 7.0:
                    wavelengths = [1931.02, 2478.77]
                    if temp == 7.5 or temp == 7.0:
                        wavelengths = [1751.98] + wavelengths
                elif a == 6.5 or a == 6.0 or a == 5.5 or a == 5.0:
                    wavelengths = [1751.98, 1993.81, 2478.77]
                    if a == 5.0: 
                        wavelengths.append(2583.08)
                elif a == 4.5 or a == 4.0 or a == 3.5: 
                    wavelengths = [1993.81, 2298.33, 2478.77, 2583.08]
                    if a == 4.0 or a == 3.5:
                        wavelengths.append(2967.40)
                elif a <= 3.0: 
                    wavelengths = [1993.81, 2409.61, 2478.77, 2583.08, 2967.40]
            elif temp == 11000:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 5.0 or a == 4.5:
                    wavelengths = [1751.98, 1993.81, 2478.77, 2583.08]
                elif a <= 4.0: 
                    wavelengths = [1993.81, 2298.33, 2478.77, 2583.08, 2967.40]
            elif temp == 12000:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a <= 5.0:
                    wavelengths = [1751.98, 1993.81, 2478.77, 2583.08]
            elif temp == 13000:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 5.0 or a == 4.5:
                    wavelengths = [1751.98, 1931.02, 1993.81, 2478.77]
                elif a <= 4.0:
                    wavelengths = [1751.98, 1993.81, 2478.77, 2583.08]
            elif temp == 14000:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 5.0 or a == 4.5:
                    wavelengths = [1751.98, 1931.02, 2478.77]
                elif a == 4.0 or a == 3.5:
                    wavelengths = [1751.98, 1931.02, 1993.81, 2478.77]
                elif a <= 3.0:
                    wavelengths = [1751.98, 1931.02, 1993.81, 2478.77, 2583.08]
            elif temp == 15000:
                if a == 9.5 or a == 9.0 or a == 8.5:
                    wavelengths = [1931.02]
                elif a == 5.0 or a == 4.5:
                    wavelengths = [1751.98, 1931.02, 2478.77]
                elif a <= 4.0:
                    wavelengths = [1751.98, 1931.02, 1993.81, 2478.77]
            for w in wavelengths:
                Resolutions.append(r)
                Temperatures.append(temp)
                Abundances.append(-a)
                Wavelengths.append(w)
                SNRs.append(np.round(float(MySpec.calc_snr_from_t(1000, w)), 3))
                ExpTimes.append(np.round((float(MySpec.calc_t_from_snr(30, w))), 3))
       
    # Temperatures from: 8,000 K to 16,000 K for DZ Stars
    for temp in range(8000, 18000, 2000):
        abundances = [7.5, 8.5]

        for a in abundances:
            MySource = PointSource()
    
            spectral_type="DZ"
            wavelength, flux = read_spectra(spectral_type, temp, a)
    
            MySource.set_spectrum(wavelengths = wavelength, spectrum = flux, unit = 'fnu')
            MySource.norm_to_AB_mag(10)

            MySpec = UVMOS_Spectroscopy(MyTelescope, MySource, MyBackground)
            
            MySpec.dispersion = r * u.nm
    
            MySpec.specify_slit()

            MySpec.calc_source_CASTORSpectrum(extraction_width=1, extraction_lowerlim=4, extraction_upperlim = 7)
            MySpec.calc_background_CASTORSpectrum(extraction_width=1, extraction_lowerlim=4, extraction_upperlim = 7)
    
            # Calculate the SNR and Exposure Times for approximate wavelengths
            if temp == 16000:
                wavelengths = [2216.436, 2055.415, 2439.05, 2802.75]
                for w in wavelengths:
                    Resolutions1.append(r)
                    Temperatures1.append(temp)
                    Abundances1.append(-a)
                    Wavelengths1.append(w)
                    SNRs1.append(np.round(float(MySpec.calc_snr_from_t(1000, w)), 3))
                    ExpTimes1.append(np.round((float(MySpec.calc_t_from_snr(30, w))), 3)) 
            elif temp == 8000:
                wavelengths = [1901.08, 2055.415, 2439.05, 2802.75]
                for w in wavelengths:
                    Resolutions1.append(r)
                    Temperatures1.append(temp)
                    Abundances1.append(-a)
                    Wavelengths1.append(w)
                    SNRs1.append(np.round(float(MySpec.calc_snr_from_t(1000, w)), 3))
                    ExpTimes1.append(np.round((float(MySpec.calc_t_from_snr(30, w))), 3)) 
            else:
                wavelengths = [1901.08, 2055.415, 2216.436, 2439.05, 2802.75]
                for w in wavelengths:
                    Resolutions1.append(r)
                    Temperatures1.append(temp)
                    Abundances1.append(-a)
                    Wavelengths1.append(w)
                    SNRs1.append(np.round(float(MySpec.calc_snr_from_t(1000, w)), 3))
                    ExpTimes1.append(np.round((float(MySpec.calc_t_from_snr(30, w))), 3))
               
    DQ_Stars = {"Resolution": Resolutions, "Temperature (K)": Temperatures, "Abundance (log C/He)": Abundances, "Wavelength (Angstom)": Wavelengths, "SNR": SNRs, "Exposure Time (sec)": ExpTimes}
    DQ_Stars = pd.DataFrame(data=DQ_Stars)
    DQ_Stars
    
    DZ_Stars = {"Resolution": Resolutions1, "Temperature (K)": Temperatures1, "Abundance (log Ca/He)": Abundances1, "Wavelength (Angstom)": Wavelengths1, "SNR": SNRs1, "Exposure Time (sec)": ExpTimes1}
    DZ_Stars = pd.DataFrame(data=DZ_Stars)

In [22]:
DQ_Stars

,Resolution,Temperature (K),Abundance (log C/He),Wavelength (Angstom),SNR,Exposure Time (sec)
0,0.061,6500,-4.0,1993.81,3.294,10449.377
1,0.061,6500,-4.0,2409.61,156.698,136.006
2,0.061,6500,-4.0,2478.77,9.941,3239.527
3,0.061,6500,-4.0,2583.08,369.300,38.022
4,0.061,6500,-4.0,2967.40,319.570,48.272
...,...,...,...,...,...,...
604,0.061,15000,-5.0,1931.02,249.908,71.127
605,0.061,15000,-5.0,2478.77,716.471,11.656
606,0.061,15000,-8.5,1931.02,341.482,43.311
607,0.061,15000,-9.0,1931.02,346.338,42.313


In [23]:
DZ_Stars

,Resolution,Temperature (K),Abundance (log Ca/He),Wavelength (Angstom),SNR,Exposure Time (sec)
0,0.061,8000,-7.5,1901.080,6.036,5527.123
1,0.061,8000,-7.5,2055.415,37.630,775.799
2,0.061,8000,-7.5,2439.050,210.197,92.316
3,0.061,8000,-7.5,2802.750,108.798,225.387
4,0.061,8000,-8.5,1901.080,33.772,876.974
5,0.061,8000,-8.5,2055.415,56.370,492.832
6,0.061,8000,-8.5,2439.050,280.028,59.640
7,0.061,8000,-8.5,2802.750,123.949,190.712
8,0.061,10000,-7.5,1901.080,72.611,368.138
9,0.061,10000,-7.5,2055.415,86.507,298.080


In [20]:
with pd.ExcelWriter('CASTOR UVMOS - White Dwarf Analysis - Test.xlsx') as writer:
    DQ_Stars.to_excel(writer, sheet_name='DQ Stars', index=False)
    DZ_Stars.to_excel(writer, sheet_name='DZ Stars', index=False)